In [ ]:
## Install and Import Dependencies 

In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118


In [3]:
!pip install transformers requests beautifulsoup4 pandas numpy

Defaulting to user installation because normal site-packages is not writeable


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [5]:
import torch 
import requests
from bs4 import BeautifulSoup 
import re

In [ ]:
## Instantiate the Model 

In [6]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
## Encode and Calculate Sentiment 

In [7]:
tokens = tokenizer.encode('Ambience was pathetic, but the food could have been good', return_tensors='pt')

In [8]:
result = model(tokens)

In [9]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.9346,  1.7114,  1.2516, -0.6504, -2.6920]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [10]:
int(torch.argmax(result.logits))+1

2

In [ ]:
## Collect Reviews

In [11]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [12]:
reviews[0]

'Hotter than I imagine myself to be and firing up the flaming ring this place packs a firey heat punch with a delicious dolls mexican fusion. Pants free or margarita mash this is the place to explore.'

In [ ]:
## Load Reviews into DataFrame and Score

In [13]:
import pandas as pd
import numpy as np

In [14]:
sentiment = pd.DataFrame(np.array(reviews), columns=['review'])

In [15]:
sentiment

,review
0,Hotter than I imagine myself to be and firing ...
1,The food and service here was really good. It...
2,Visiting from Texas and decided to give this r...
3,"Great atmosphere, attentive service, solid mar..."
4,Don't come here expecting legit Mexican food b...
5,Out of all the restaurants that I tried in Syd...
6,We came here on a Thursday night @ 5pm and by ...
7,The food is fresh and tasty. The scallop cevi...
8,Have been here twice and have absolutely loved...
9,I was pleasantly surprised at what a great job...


In [16]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [17]:
sentiment['sentiment_score'] = sentiment['review'].apply(lambda x: sentiment_score(x[:512]))

In [18]:
sentiment

,review,sentiment_score
0,Hotter than I imagine myself to be and firing ...,5
1,The food and service here was really good. It...,5
2,Visiting from Texas and decided to give this r...,5
3,"Great atmosphere, attentive service, solid mar...",3
4,Don't come here expecting legit Mexican food b...,3
5,Out of all the restaurants that I tried in Syd...,5
6,We came here on a Thursday night @ 5pm and by ...,4
7,The food is fresh and tasty. The scallop cevi...,4
8,Have been here twice and have absolutely loved...,5
9,I was pleasantly surprised at what a great job...,5
